# Modern and functional python to build robust software

*Following the conference By Guillaume Desforges*

This Notebook contains some samples of code. They are used to illustrate My blog article that summarizes the conferences I attended at the (very great) Pycon Fr 2023, in Bordeaux, France.
The code comes from the one showed at the conference in Guillaume's slides with some personal comments. It may contain very small modifications. It is not all the code shown at the presentation.

## 1. Pure and impure functions

Let's define a funtion, in several ways:

In [1]:
# This function looks nice:


def increment(x):
    return x + 1


increment(1)

2

In [2]:
# This one looks not so good:


I = 1


def increment(x):
    return x + I


increment(1)

2

The problem with the latter function is that if we modify the value of I, we will also modify the result of the function when we give him the exact same parameter:

In [3]:
print(f"When I is 1, increment(1) is: {increment(1)}")

I = 0
print(f"When I is 1, increment(1) is: {increment(1)}")

When I is 1, increment(1) is: 2
When I is 1, increment(1) is: 1


Not only this is bad, but it also can pass unit testing :

In [4]:
import ipytest
import pytest

ipytest.autoconfig()

In [5]:
%%ipytest
I = 1


# This will pass the test because I is currently 1, but it may not be when the function is called
def test_increment():
    assert increment(1) == 2

.                                                                                            [100%]
1 passed in 0.00s


We could also write the function like this:

In [6]:
def increment(x):
    result = x + 1

    with open("increments.txt", mode="a") as f:
        f.write(f"{result}\n")
    return result

This function has a big problem in it, the fact that we are writing the result of the function in a file is whais a called a "side effect". If the file is not accessible, the function will crash and I will not return the result.

### Onion architecture

The demonstration used for this part of the conference is based on a Flask Application.
Let's pretend we have this code, we are not going to try to run it because we are not launching flask app:

```
app/web.py
``` 


```python
import json

from flask import Flask, request
from flask_sqlalchemy import SQLAlchemy

with open("config.json") as f:
    data = json.load(f)

DISCOUNT = float(json.load("config.json")["discount"])

db = SQLAlchemy()


class Sale(db.Model):
    price = db.Column(db.Float)


app = Flask()
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///project.db"
db.init_app(app)


@app.get("/discount")
def get_discount():
    return DISCOUNT


@app.post("/sales")
def create_sale():
    price = float(request.args.price)
    
    # Original slide had it as 'final_price *= 1 - DISCOUNT', I think it is a typo:
    final_price = price - DISCOUNT 
    db.session.add(Sale(final_price))
    db.session.commit()
    return 200


if __name__ == "__main__":
    app.run()
```

---
There is a problem with this statement:

```python
DISCOUNT = float(json.load("config.json")["discount"])
```
---
It can give a problem every time we launch the application if there is a problem with the file or with its content.

Another problem is, if we have the following code, when we run it from the CLI (Command Line Interface), if we have a problem with the json file, we can not use the function, and this is not good because we do not even need to have a running application to do it :
```python
from app.web import db, Sale

print(sum(sale.price for sale in db.session.execute(db.select(Sale))))
```


We can replace it with the following, 

```python
def read_discount():
    return float(json.load("config.json")["discount"])


@app.get("/discount")
def get_discount():
    return read_discount()


@app.post("/sales")
def create_sale():
    price = float(request.args.price)
    final_price *= 1 - read_discount() #<-- we call the function here 
    db.session.add(Sale(final_price))
    db.session.commit()
    return 200
```

We can still get a problem if we call read_discount() from another function, for example, if we create the following, which gives us the discounted price from a given (original) price:

```python
def compute_final_price(price):
    return price * (1 - read_discount()) # remember, read_discount calls the json file
```

This function calls read_discount() which is an impure function, and it becomes impure too. If we want to unit test it, we need to have a json file with all the proper info in it. We would be better of if we transform the impure function into a pure function, and we can eventually use the result of the impure function as an argument when we execute it in our program.

```python
def compute_final_price(price, discount):
    return price * (1 - discount()) # this is a pure function now
```

And our flask function now looks like this:

```python
import json

from flask import Flask, request
from flask_sqlalchemy import SQLAlchemy

with open("config.json") as f:
    data = json.load(f)

DISCOUNT = float(json.load("config.json")["discount"])

db = SQLAlchemy()


class Sale(db.Model):
    price = db.Column(db.Float)


app = Flask()
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///project.db"
db.init_app(app)


@app.get("/discount")
def get_discount():
    return DISCOUNT


@app.post("/sales")
def create_sale():
    price = float(request.args.price)
    final_price *= compute_final_price(price, discount)
    db.session.add(Sale(final_price))
    db.session.commit()
    return 200


if __name__ == "__main__":
    create_app.run()
```

What we did is to make the impurity be in the outer layers of the application, we now have pure functions that we can test separately and that do not depend on the call to the json file.

## 2. Immutability

We can define a classes using the ```@dataclass``` decorator. 

In the example below, we define the same class twice, once without the ```frozen=True``` parameter, which leads to a mutable class, and the second one uses the parameter to make the class immutable.

In [7]:
from dataclasses import dataclass

In [8]:
@dataclass
class Point:
    x: int
    y: int

    def __hash__(self):
        return self.x << 16 | self.y


p = Point(x=1, y=0)
d = {p: "found"}

print(p == Point(x=1, y=0))  # True
print(d[Point(x=1, y=0)])  # found

p.x = 2  # We change the value of attribute x in the object p

print(p)  ## Point(x=2, y=0)
print(p == Point(x=2, y=0))  # True
print(d[Point(x=2, y=0)])  # KeyError!

True
found
Point(x=2, y=0)
True


KeyError: Point(x=2, y=0)

In [9]:
# We use frozen=True to make the class immutable
@dataclass(frozen=True)
class Point:
    x: int
    y: int

    def __hash__(self):
        return self.x << 16 | self.y


p = Point(x=1, y=0)
d = {p: "found"}

print(p == Point(x=1, y=0))  # True
print(d[Point(x=1, y=0)])  # found

p = Point(x=2, y=0)  # This works!
print(p == Point(x=1, y=0))  # Now this is False

p.x = 2  # FrozenInstanceError!

True
found
False


FrozenInstanceError: cannot assign to field 'x'

We can also define immutable class using the ```@property``` decorator:

In [10]:
class Thing:
    def __init__(self, a):
        self._a = a

        @property
        def a(self):
            return self._a

        @a.setter
        def a(self, value):
            raise AttributeError(f"{self} is immutable")

This methods are good practices, but they do not solve the core problem, plus there is nothing we can do if we use mutable types, like lists or dictionaries.

So we can change the perspective and think that it is our responsability as coders to not mutate the variables that we introduce in our functions. A way to this is by using ```deepcopy()```:

In [11]:
from copy import deepcopy


def f(x):
    x = deepcopy()
    return g(
        x
    )  # g is not defined here, but since the new x is a copy, g can not change the "global" x

## 3. Working with types (Static Typing with python)

If we define the following f(x) function, that returns x.lower(), it will crash if we introduce anything else than a string as a parameter:

In [12]:
def f(x):
    return x.lower()


def main():
    f("Hello")  # This works
    f(1)  # This cxrashes (it is not a string, it does not have .lower() method)

In [13]:
main()

AttributeError: 'int' object has no attribute 'lower'

But we can add static types to the function definition:

In [14]:
def f(x: str) -> str:
    return x.lower()

This is not enough by itself, if we do not use mypy or pyright, the types we assigned are sole indications:

In [15]:
f(1)

AttributeError: 'int' object has no attribute 'lower'

Also, if we do not use type checkers, we totally can write absurd things such as:

In [16]:
def f(x: int) -> dict:
    return x.lower()


f("ABCD")  # It will work!

'abcd'